In [4]:
from PIL import Image




import xml.etree.ElementTree as ET
import os

def convert_xml_to_yolo(xml_file, image_width, image_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    with open(xml_file.replace('.xml', '.txt'), 'w') as f:
        for obj in root.findall('object'):
            name = obj.find('name').text
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # Normalize the bounding box coordinates
            x_center = (xmin + xmax) / 2 / image_width
            y_center = (ymin + ymax) / 2 / image_height
            width = (xmax - xmin) / image_width
            height = (ymax - ymin) / image_height

            # Assuming license plate is class '0'
            f.write(f"0 {x_center} {y_center} {width} {height}\n")

def process_images(image_folder):
    for filename in os.listdir(image_folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".JPG",".png",".PNG")):
            image_path = os.path.join(image_folder, filename)
            xml_path = os.path.join(image_folder, filename.replace(os.path.splitext(filename)[1], ".xml"))
            image=Image.open(image_path)

            image_width, image_height = image.size  # Set according to your image size
            convert_xml_to_yolo(xml_path, image_width, image_height)

#if __name__ == "__main__":
#    process_images('dataset/images/', 'dataset/annotations/')


In [5]:
process_images("dataset/google_images/")

process_images("dataset/video_images/")

In [6]:
import os
import pathlib
def iterate_folders(parent_folder):
    # Loop through all entries in the parent folder
    for folder_name in os.listdir(parent_folder):
        # Check if the entry is a directory (folder)
        folder_path = os.path.join(parent_folder, folder_name)
        if os.path.isdir(folder_path):
            print("Folder name:", folder_name)
            process_images(pathlib.Path(parent_folder,folder_name))

# Example usage
parent_folder = 'dataset/State-wise_OLX/'  # Replace with your actual path
iterate_folders(parent_folder)


Folder name: AN
Folder name: AP
Folder name: AR
Folder name: AS
Folder name: BR
Folder name: CG
Folder name: CH
Folder name: DL
Folder name: DN
Folder name: GA
Folder name: GJ
Folder name: HP
Folder name: HR
Folder name: JH
Folder name: JK
Folder name: KA
Folder name: KL
Folder name: LA
Folder name: MH
Folder name: ML
Folder name: MN
Folder name: MP
Folder name: MZ
Folder name: NL
Folder name: OD
Folder name: PB
Folder name: PY
Folder name: RJ
Folder name: SK
Folder name: TN
Folder name: TR
Folder name: TS
Folder name: UK
Folder name: UP
Folder name: WB


In [7]:
i = "dataset/google_images/00b42b2c-f193-4863-b92c-0245cbc816da___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano-Petrol-Review-Images-Black-Front-Angle.jpg"

In [ ]:
from PIL import Image

# Open the image using PIL
image = Image.open(i)

# Get image details (size, mode)
width, height = image.size
print(f"Image Dimensions: {width}x{height}")
print(f"Image Mode: {image.mode}")  # RGB, L, etc.

# Display the image
display(image)


In [ ]:
import cv2
from matplotlib import pyplot as plt

# Load the image using OpenCV (BGR format)
image = cv2.imread(i)

# Convert to RGB (Matplotlib uses RGB, OpenCV uses BGR)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Get image details (height, width, channels)
height, width, channels = image.shape
print(f"Image Dimensions: {height}x{width}")
print(f"Number of Channels: {channels}")

# Display the image
plt.imshow(image_rgb)
plt.axis('off')  # Turn off axis numbers and ticks
plt.show()


In [3]:
from ultralytics import YOLO
print("✅ YOLO module imported successfully!")


✅ YOLO module imported successfully!


In [2]:
!pip install ultralytics


In [ ]:
from ultralytics import YOLO
print(YOLO('yolov8n.pt'))  # Should print model details if installed correctly


In [ ]:
import os
import shutil
import random
from tqdm import tqdm


In [10]:
import os
import pathlib
def iterate_folders(parent_folder):
    # Loop through all entries in the parent folder
    for folder_name in os.listdir(parent_folder):
        # Check if the entry is a directory (folder)
        folder_path = os.path.join(parent_folder, folder_name)
        if os.path.isdir(folder_path):
            dataset_path = pathlib.Path(parent_folder,folder_name) 
            output_dir = "dataset/split"  # Destination for train/val split
            train_ratio=0.8
            train_img_dir = os.path.join(output_dir, "train/images")
            train_label_dir = os.path.join(output_dir, "train/labels")
            val_img_dir = os.path.join(output_dir, "val/images")
            val_label_dir = os.path.join(output_dir, "val/labels")

            os.makedirs(train_img_dir, exist_ok=True)
            os.makedirs(train_label_dir, exist_ok=True)
            os.makedirs(val_img_dir, exist_ok=True)
            os.makedirs(val_label_dir, exist_ok=True)
            image_extensions = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")

# Collect image and label pairs
            image_files = [f for f in os.listdir(dataset_path) if f.endswith(image_extensions)]
            random.shuffle(image_files)  # Shuffle dataset

            split_idx = int(len(image_files) * train_ratio)
            train_files = image_files[:split_idx]
            val_files = image_files[split_idx:]
            def move_files(file_list, dest_img_dir, dest_label_dir):
                  for file in tqdm(file_list, desc=f"Moving files to {dest_img_dir}"):
                         img_src = os.path.join(dataset_path, file)
                         label_src = os.path.join(dataset_path, file.replace(os.path.splitext(file)[1], ".txt"))

                         img_dest = os.path.join(dest_img_dir, file)
                         label_dest = os.path.join(dest_label_dir, file.replace(os.path.splitext(file)[1], ".txt"))

                         shutil.copy(img_src, img_dest)
                         if os.path.exists(label_src):  # Copy label only if it exists
                             shutil.copy(label_src, label_dest)

# Move train and validation files
            move_files(train_files, train_img_dir, train_label_dir)
            move_files(val_files, val_img_dir, val_label_dir)





# Example usage
parent_folder = 'dataset/State-wise_OLX/'  # Replace with your actual path
iterate_folders(parent_folder)


Moving files to dataset/split\val/images: 100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 333.09it/s]
Moving files to dataset/split\train/images: 0it [00:00, ?it/s]
Moving files to dataset/split\val/images: 100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 370.98it/s]
Moving files to dataset/split\train/images: 0it [00:00, ?it/s]
Moving files to dataset/split\val/images: 100%|█████████████████████████████████████████| 5/5 [00:00<00:00, 416.72it/s]


In [15]:
dataset_path = "dataset/google_images"  # Your dataset folder containing images and XML
output_dir = "dataset/split"  # Destination for train/val split

train_ratio = 0.8  # 80% train, 20% validation


In [16]:
train_img_dir = os.path.join(output_dir, "train/images")
train_label_dir = os.path.join(output_dir, "train/labels")
val_img_dir = os.path.join(output_dir, "val/images")
val_label_dir = os.path.join(output_dir, "val/labels")

os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)


In [17]:
image_extensions = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")

# Collect image and label pairs
image_files = [f for f in os.listdir(dataset_path) if f.endswith(image_extensions)]
random.shuffle(image_files)  # Shuffle dataset

split_idx = int(len(image_files) * train_ratio)
train_files = image_files[:split_idx]
val_files = image_files[split_idx:]


In [18]:
def move_files(file_list, dest_img_dir, dest_label_dir):
    for file in tqdm(file_list, desc=f"Moving files to {dest_img_dir}"):
        img_src = os.path.join(dataset_path, file)
        label_src = os.path.join(dataset_path, file.replace(os.path.splitext(file)[1], ".txt"))

        img_dest = os.path.join(dest_img_dir, file)
        label_dest = os.path.join(dest_label_dir, file.replace(os.path.splitext(file)[1], ".txt"))

        shutil.copy(img_src, img_dest)
        if os.path.exists(label_src):  # Copy label only if it exists
            shutil.copy(label_src, label_dest)

# Move train and validation files
move_files(train_files, train_img_dir, train_label_dir)
move_files(val_files, val_img_dir, val_label_dir)



Moving files to dataset/split\val/images: 100%|███████████████████████████████████████| 89/89 [00:00<00:00, 291.11it/s]


In [50]:
'''MOVED ORIGINAL DATSSETS FILE FROM DATASET TO ORIGINAL_DATASET TO HAVE CORREC STRUCTURE FOR DATASET FOLDER HAVINHG FILES FOR TRAIN-VAL'''

'MOVED ORIGINAL DATSSETS FILE FROM DATASET TO ORIGINAL_DATASET TO HAVE CORREC STRUCTURE FOR DATASET FOLDER HAVINHG FILES FOR TRAIN-VAL'

In [10]:
!pip install albumentations ultralytics opencv-python tqdm


In [20]:
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define Augmentation Pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=15, p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.2),
    A.CLAHE(p=0.1),
], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))


def read_yolo_annotations(label_path):
    """Reads YOLO annotations and returns bounding boxes & class labels (normalized)."""
    boxes, class_labels = [], []
    try:
        with open(label_path, "r") as file:
            lines = file.readlines()
            if not lines:
                print(f"Warning: {label_path} is empty. Skipping.")
                return [], []

            for line in lines:
                parts = line.strip().split()
                if len(parts) < 5:  # Ensure valid YOLO format
                    print(f"Invalid annotation in {label_path}: {line}")
                    continue

                class_id = int(parts[0])
                bbox = list(map(float, parts[1:]))  # Normalized [x_center, y_center, width, height]
                boxes.append(bbox)
                class_labels.append(class_id)
    except Exception as e:
        print(f"Error reading {label_path}: {e}")
    return boxes, class_labels


def write_yolo_annotations(output_path, boxes, class_labels):
    """Writes augmented YOLO annotations (already normalized)."""
    try:
        with open(output_path, "w") as file:
            for cls, bbox in zip(class_labels, boxes):
                file.write(f"{cls} {' '.join(map(str, bbox))}\n")
    except Exception as e:
        print(f"Error writing {output_path}: {e}")


def clip_bboxes(bboxes):
    """Clips bounding box coordinates to ensure they are in the range [0, 1]."""
    clipped_bboxes = []
    for bbox in bboxes:
        x, y, w, h = bbox

        # Ensure values are within the valid range
        x = np.clip(x, 0, 1)
        y = np.clip(y, 0, 1)
        w = np.clip(w, 0, 1)
        h = np.clip(h, 0, 1)

        # Skip invalid boxes
        if w == 0 or h == 0:
            print(f"Skipping invalid bbox: {bbox}")
            continue

        clipped_bboxes.append([x, y, w, h])
    return clipped_bboxes


def augment_image(image_path, label_path, output_folder, num_augmentations=8):
    """Applies multiple augmentations per image and saves the results."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Failed to load image {image_path}. Skipping.")
        return

    height, width, _ = image.shape

    # Read YOLO annotations
    boxes, class_labels = read_yolo_annotations(label_path)
    if not boxes:
        print(f"Skipping {image_path} due to missing or invalid bounding boxes.")
        return

    clipped_boxes = clip_bboxes(boxes)

    for i in range(num_augmentations):
        try:
            augmented = transform(image=image, bboxes=clipped_boxes, class_labels=class_labels)
            aug_bboxes = augmented["bboxes"]

            # Skip augmentation if bounding boxes are missing after transformation
            if not aug_bboxes:
                print(f"Skipping augmentation {i} for {image_path} due to bbox issue.")
                continue

            # Save augmented image
            aug_img_name = os.path.basename(image_path).replace(".jpg", f"_aug{i}.jpg")
            cv2.imwrite(os.path.join(output_folder, aug_img_name), augmented["image"])

            # Save new annotations
            aug_label_name = os.path.basename(label_path).replace(".txt", f"_aug{i}.txt")
            write_yolo_annotations(os.path.join(output_folder, aug_label_name), aug_bboxes, class_labels)

        except ValueError as v:
            print(f"Augmentation error for {image_path}, iteration {i}: {v}")
            print(f"Bounding boxes: {clipped_boxes}")
            continue


# Directory Paths
image_folder = "dataset/split/train/images/"
label_folder = "dataset/split/train/labels/"
output_folder = "dataset/split/augmented/"

os.makedirs(output_folder, exist_ok=True)

# Define supported image extensions
image_extensions = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")

# Collect image and label pairs
image_files = [f for f in os.listdir(image_folder) if f.endswith(image_extensions)]

# Process All Images
for idx, image_file in enumerate(image_files):
    image_path = os.path.join(image_folder, image_file)
    label_path = os.path.join(label_folder, image_file.rsplit('.', 1)[0] + ".txt")
   
    if os.path.exists(label_path):
        augment_image(image_path, label_path, output_folder)
    else:
        print(f"Warning: No label file found for {image_file}. Skipping.")

print("Augmentation completed! ✅")


Augmentation error for dataset/split/train/images/HP20.jpg, iteration 0: Expected y_max for bbox [0.16176471 0.9444444  0.36029413 1.0555556  0.        ] to be in the range [0.0, 1.0], got 1.0555555820465088.
Bounding boxes: [[0.2610294117647059, 1.0, 0.19852941176470587, 0.1111111111111111]]
Augmentation error for dataset/split/train/images/HP20.jpg, iteration 1: Expected y_max for bbox [0.16176471 0.9444444  0.36029413 1.0555556  0.        ] to be in the range [0.0, 1.0], got 1.0555555820465088.
Bounding boxes: [[0.2610294117647059, 1.0, 0.19852941176470587, 0.1111111111111111]]
Augmentation error for dataset/split/train/images/HP20.jpg, iteration 2: Expected y_max for bbox [0.16176471 0.9444444  0.36029413 1.0555556  0.        ] to be in the range [0.0, 1.0], got 1.0555555820465088.
Bounding boxes: [[0.2610294117647059, 1.0, 0.19852941176470587, 0.1111111111111111]]
Augmentation error for dataset/split/train/images/HP20.jpg, iteration 3: Expected y_max for bbox [0.16176471 0.9444444

In [11]:
import yaml 
from pathlib import Path

In [12]:
dataset_path=Path("dataset")
data_yaml_path=dataset_path/"data.yaml"

In [13]:
# Write data.yaml
if not data_yaml_path.exists():
    data_yaml = {
        "train": str(dataset_path / "images/train"),
        "val": str(dataset_path / "images/val"),
        "nc": 1,  # Number of classes
        "names": ["plate"]  # Change according to your dataset
    }
    with open(data_yaml_path, "w") as f:
        yaml.dump(data_yaml, f, default_flow_style=False)
        
print("✅ Dataset setup completed!")


✅ Dataset setup completed!


In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Count:", torch.cuda.device_count())
print("Current Device:", torch.cuda.current_device())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


CUDA Available: False
GPU Count: 0


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should show number of GPUs


In [14]:
import pathlib

In [6]:
from ultralytics import YOLO
import torch


# Initialize the YOLOv8 model
model = YOLO('yolov8n.pt')  # You can change to another pre-trained model if needed

# Start training
model.train(
    imgsz=640,  # Correct argument for image size
    batch=8,   # Correct argument for batch size
    epochs=100,
    data="data.yaml",
    pretrained='yolov8n.pt',
    device=0,  # Dynamically set the device to 'cuda' or 'cpu'
    save=True,
    name="trained1"
)


C:\Users\dell\anaconda3\envs\vlpr-env\lib\site-packages\ultralytics\nn\tasks.py:511: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Should return True if CUDA is enabled

!python -c "import torchvision; print(torchvision.__version__)"


In [ ]:
import torchvision
print(torchvision.__version__)

# Check if torchvision has NMS
try:
    from torchvision.ops import nms
    print("NMS is available in torchvision.")
except ImportError:
    print("NMS is NOT available in torchvision.")


In [6]:
import psutil
ram = psutil.virtual_memory()
print(f"Total RAM: {ram.total / 1e9:.2f} GB")
print(f"Available RAM: {ram.available / 1e9:.2f} GB")
print(f"Used RAM: {ram.used / 1e9:.2f} GB")
print(f"RAM Usage: {ram.percent}%")


Total RAM: 16.85 GB
Available RAM: 7.02 GB
Used RAM: 9.83 GB
RAM Usage: 58.4%


In [9]:
import os
import psutil

pid = os.getpid()  # Get current Jupyter Notebook process ID
memory_usage = psutil.Process(pid).memory_info().rss / 1e9  # Convert to GB
print(f"Jupyter is using: {memory_usage:.2f} GB of RAM")


Jupyter is using: 0.44 GB of RAM


In [8]:
import gc
import torch

# Collect garbage to free up memory
gc.collect()

# If using GPU, clear the cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()  # This line is GPU-specific, can be omitted for CPU

# Optional: Reset memory cache if you're using PyTorch and running on CPU
torch.cuda.empty_cache()  # Clears the PyTorch cache for both GPU and CPU if needed


In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available. Number of devices: {torch.cuda.device_count()}")
    
    # Check the name of each device and its corresponding index (0, 1, etc.)
    for i in range(torch.cuda.device_count()):
        print(f"CUDA Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")


CUDA is available. Number of devices: 1
CUDA Device 0: NVIDIA GeForce RTX 3050 6GB Laptop GPU
